# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121451e+02     1.741634e+00
 * time: 0.09807300567626953
     1     1.033478e+01     9.307798e-01
 * time: 0.9714679718017578
     2    -1.145460e+01     1.061234e+00
 * time: 1.110741138458252
     3    -3.402272e+01     8.192082e-01
 * time: 1.2877051830291748
     4    -4.734896e+01     5.363691e-01
 * time: 1.448047161102295
     5    -5.682760e+01     2.290778e-01
 * time: 1.6009809970855713
     6    -5.965222e+01     1.614126e-01
 * time: 1.7137880325317383
     7    -6.081500e+01     7.973542e-02
 * time: 1.8350720405578613
     8    -6.133630e+01     4.212304e-02
 * time: 1.956305980682373
     9    -6.165339e+01     3.057895e-02
 * time: 2.0697901248931885
    10    -6.180872e+01     2.489596e-02
 * time: 2.190592050552368
    11    -6.194930e+01     2.378512e-02
 * time: 2.3032779693603516
    12    -6.202537e+01     2.047869e-02
 * time: 2.422790050506592
    13    -6.207825e+01     1.886690e-02
 * time: 2.54565715789794

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671080
    AtomicLocal         -18.8557662
    AtomicNonlocal      14.8522614
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336825

    total               -62.261666457692
